In [1]:
import pandas as pd
import os
data = pd.read_csv("../../data/retail_sales/marketing.csv")
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1826,1970,Graduation,Divorced,"$84,835.00",0,0,6/16/14,0,189,...,6,1,0,0,0,0,0,1,0,SP
1,1,1961,Graduation,Single,"$57,091.00",0,0,6/15/14,0,464,...,7,5,0,0,0,0,1,1,0,CA
2,10476,1958,Graduation,Married,"$67,267.00",0,1,5/13/14,0,134,...,5,2,0,0,0,0,0,0,0,US
3,1386,1967,Graduation,Together,"$32,474.00",1,1,5/11/14,0,10,...,2,7,0,0,0,0,0,0,0,AUS
4,5371,1989,Graduation,Single,"$21,474.00",1,0,4/8/14,0,6,...,2,7,1,0,0,0,0,1,0,SP


In [2]:
data['ID'].unique().shape[0]==data.shape[0]

True

In [3]:
data.isnull().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
 Income                24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Response                0
Complain                0
Country                 0
dtype: int64


### Tentative Ideas and Questions
Ref [Framework](https://docs.google.com/spreadsheets/d/1rzPZhRoqIfZkTltYQ429492U_sSHM5BLICL6E4sPau4/edit#gid=528193782)

- **Input file handling**: 
    - We can break down the file into two files, one containing the demographics and the other containing the campaign data.

    - Apart from breaking down into two files, we can also change the format, keep one a flat file (csv,tsv,etc) and another file as json or xml.

- **Data Quality Checks**: 
    - Some columns don't have appropriate datatypes (Income, Dt_Customer), make business questions around them so that these have to be handled
    - Nothing else is very apparent
    
- **EDA**
    - Univariates for prominent categorical(marital status,education, acceptedcmp1,2,3,4) and continous variables(income). Build business questions around these?
    - Extereme values for income, amount spent on different categories, recency etc might be interesting to look at.

- **Data Analysis**
    - Generate and check hypothesis around Amount Spent on different categories and response in different marketing campaigns
    - Profile analysis of people who respond vs who don't 
    - Do people who accept the offer in first campaign also accept in any other campaign?
    - Create a funnel analysis showing what percentage of unique customers accept campaign 1,2,3,..etc
    - Find out how income impacts the amount spent on
        - Wine
        - Meat Products
        - Gold Products
        - Fish Products
    - Can you test the hypothesis that recent customers complain less in general compared to older customers

### Break data into non standard formats
- Create a demographics file (tsv)
- Create a buying behaviour file (json)
- Create a campaign response file (xml)

In [4]:
demo_cols = ['ID','Year_Birth','Education','Marital_Status',' Income ','Kidhome','Teenhome','Dt_Customer','Country']
demographics = data[demo_cols].copy()

behaviour_cols = ['ID','Recency','MntWines','MntFruits','MntMeatProducts','MntFishProducts','MntSweetProducts','MntGoldProds','NumDealsPurchases','NumWebPurchases','NumCatalogPurchases','NumStorePurchases','NumWebVisitsMonth']
behaviour = data[behaviour_cols].copy()

campaign_cols = ['ID','AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response','Complain']
campiagn = data[campaign_cols].copy()

In [5]:
data_dict = []
for idx,row in enumerate(behaviour.itertuples()):
    data_dict.append({f'ID_{row[1]}':
                           {
                         'Recency':row[2],
                         'MntWines':row[3],
                         'MntFruits':row[4],
                         'MntMeatProducts':row[5],
                         'MntFishProducts':row[6],
                         'MntSweetProducts':row[7],
                         'MntGoldProds':row[8],
                         'NumDealsPurchases':row[9],
                         'NumWebPurchases':row[10],
                         'NumCatalogPurchases':row[11],
                         'NumStorePurchases':row[12],
                         'NumWebVisitsMonth':row[13]}})

In [6]:
import json
with open("../../data/retail_sales/behaviour.json","w",encoding='utf-8') as file:
    json.dump(data_dict, file, ensure_ascii=False, indent=4)

In [7]:
data_dict = []
for idx,row in enumerate(campiagn.itertuples()):
    data_dict.append({f'ID_{row[1]}':
                           {
                         'AcceptedCmp1':row[2],
                         'AcceptedCmp2':row[3],
                         'AcceptedCmp3':row[4],
                         'AcceptedCmp4':row[5],
                         'AcceptedCmp5':row[6],
                         'Response':row[7],
                         'Complain':row[8]}})

In [8]:
with open("../../data/retail_sales/campaign.json","w",encoding='utf-8') as file:
    json.dump(data_dict, file, ensure_ascii=False, indent=4)

In [9]:
demographics.to_csv('../../data/retail_sales/demographics.txt',sep="\t",index=False)

In [11]:
data[['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response']].sum(axis=1)

0       1
1       2
2       0
3       0
4       2
       ..
2235    0
2236    0
2237    0
2238    0
2239    3
Length: 2240, dtype: int64